*This notebook is adapted from the [Unsloth Phi-3 Mini Training Notebook](https://colab.research.google.com/drive/1lN6hPQveB_mHSnTOYifygFcrO8C1bxq4?usp=sharing) tutorial. Go support them!*

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a><br>Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

In [ ]:
%%capture

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
apt-get install build-essential -y
pip install huggingface_hub wandb
pip uninstall torch -y
pip install torch==2.3.0
pip install -U optimum
pip uninstall xformers -y
pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
wandb login <YOUR_WANDB_API_KEY>

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1248 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct-v0-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["train.jsonl"], split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
eval_dataset = load_dataset('gsarti/eureka-rebus', 'llm_sft', data_files=["id_test.jsonl", "ood_test.jsonl"], split = "train")
eval_dataset = eval_dataset.map(formatting_prompts_func, batched = True)

In [8]:
print(dataset[5]["text"])

<s><|user|>
Risolvi gli indizi tra parentesi per ottenere una prima lettura, e usa la chiave di lettura per ottenere la soluzione del rebus.

Rebus: U [Lo è il passacavallo] LO [È fatta di vimini] F F [Sono decimi di chilo] S [Disusato soprabito] A [Un rampicante dei Tropici]
Chiave di lettura: 3 6 12 8<|end|>
<|assistant|>
Procediamo alla risoluzione del rebus passo per passo:
- U = U
- [Lo è il passacavallo] = nave
- L O = L O
- [È fatta di vimini] = cesta
- F F = F F
- [Sono decimi di chilo] = etti
- S = S
- [Disusato soprabito] = tait
- A = A
- [Un rampicante dei Tropici] = liana

Prima lettura: U nave LO cesta F F etti S tait A liana

Ora componiamo la soluzione seguendo la chiave risolutiva:
3 = Una
6 = veloce
12 = staffettista
8 = italiana

Soluzione: Una veloce staffettista italiana
<|end|>



In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

response_template = "<|assistant|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    data_collator=collator,
    args = TrainingArguments(
        per_device_train_batch_size = 32, # Tested on a 24GB RTX 3090 with 4bit quantization
        gradient_accumulation_steps = 1,
        warmup_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 150,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        num_train_epochs = 2,
        save_steps = 500,
        save_total_limit = 3,
        eval_strategy = "steps",
        eval_steps = 500,
        push_to_hub = True,
        hub_model_id = "phi3-mini-rebus-solver-adapters",
        report_to="wandb"
    ),
)

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.643 GB.
2.283 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 81,136 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 5,072
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss,Validation Loss
500,0.064500,0.076138
1000,0.039700,0.058289
1500,0.027300,0.050786
2000,0.022000,0.045427
2500,0.019200,0.043353
3000,0.009900,0.044117
3500,0.010200,0.042471
4000,0.008900,0.041404
4500,0.008300,0.040545


In [ ]:
# Save merged model in FP16
model.push_to_hub_merged("phi3-mini-rebus-solver-fp16", tokenizer, save_method = "merged_16bit")

# Save model to GGUF for local inference
model.push_to_hub_gguf("phi3-mini-rebus-solver-Q8_0-GGUF", tokenizer)